# Notebook for RT2 Assignment

In [1]:
import rospy
from geometry_msgs.msg import Point, Twist
from rosgraph_msgs.msg import Clock
from actionlib_msgs.msg import GoalID
from std_msgs.msg import Bool

import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets 
from ipywidgets import Button, Layout , interactive_output , interactive
from ipywidgets import ButtonStyle, GridBox, VBox, HBox
from IPython.display import display

from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
import rospy
import tf
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from tf.transformations import quaternion_matrix
import numpy as np

rospy.init_node("input_console")

pubGoalPoint = rospy.Publisher("MY_topic_send_goal", Point, queue_size=1)
pubCancel = rospy.Publisher("move_base/cancel", GoalID, queue_size=1)
pubTeleop = rospy.Publisher("MY_topic_teleop", Bool, queue_size=1)
pubManualDrive = rospy.Publisher("myRemapped_cmd_vel", Twist, queue_size=1)


In [2]:
my_cancel = GoalID()
inputPoint = Point()
whichTeleop = Bool()
desiredVel = Twist() 

## classes for robot visualization

In [19]:
class Odom_Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = self.ax.plot([], [], 'r.')  #0000
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        self.ax.set_title('Odometry graphic',fontsize=20)
        return self.ln
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
#         if len(self.y_data)> 50:
#             self.y_data.pop(0)
#         if len(self.y_data)> 50:
#             self.x_data.pop(0)
            
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

class Laser_Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots(subplot_kw={'projection': 'polar'})
        self.ln, = self.ax.plot([], [], 'r.')
        self.r, self.theta = [] , []
        self.rmax = 10
        
    def plot_init(self):
        self.ax.set_thetalim(-np.pi/2, np.pi/2)
        self.ax.set_rmax(self.rmax)
        self.ax.set_theta_zero_location("N")
        self.ax.set_title('Lasers Scan graphic',fontsize=20)
        return self.ln
    
    def laser_callback(self, msg):
        self.r = msg.ranges
        self.theta = np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment)
        
    def update_plot(self, frame):
        for artist in plt.gca().lines + plt.gca().collections:
            artist.remove()
        self.ln = self.ax.plot(self.theta,self.r,'r.')
        self.ax.set_rmax(self.rmax)
        return self.ln
    
class Goal_Visualiser:
    def __init__(self):
        global goal_reached, goal_NON_reached
        self.fig, self.ax = plt.subplots()
        data = {'reached': goal_reached, 'UNreached': goal_NON_reached}
        names = list(data.keys())
        values = list(data.values())
        self.x_data, self.y_data = names ,values
        self.ln = self.ax.bar(self.x_data, self.y_data, color = ['green','red']) #0000 self.ln,
        
    def plot_init(self):
#         self.ax.set_xlim(-10, 10)
        self.ax.set_ylabel('goal',fontsize=10)
        self.ax.set_title('Reached Goals Graphic',fontsize=20)
        return self.ln
    
    def goal_callback(self, msg):
        global goal_reached, goal_NON_reached
        data = {'reached': goal_reached, 'UNreached': goal_NON_reached}
        names = list(data.keys())
        values = list(data.values())
        self.x_data, self.y_data = names ,values            
        
    def update_plot(self, frame):
        self.ln = self.ax.bar(self.x_data, self.y_data, color = ['green','red'])  #0000
        return self.ln

### functions to unable/disable widgets

In [4]:


def DisableGoal(bool):
    if bool == 0:
        y_goal_widget.disabled = False
        x_goal_widget.disabled = False
    else:
        y_goal_widget.disabled = True
        x_goal_widget.disabled = True

def DisableCommands(bool):
    if bool == 0:
        b_forward.disabled = False
        b_forLeft.disabled = False
        b_forRight.disabled = False
        b_turnLeft.disabled = False
        b_turnRight.disabled = False
        b_stop.disabled = False
        b_backLeft.disabled = False
        b_backRight.disabled = False
        b_back.disabled = False
        s_speed.disabled = False
    else:
        b_forward.disabled = True
        b_forLeft.disabled = True
        b_forRight.disabled = True
        b_turnLeft.disabled = True
        b_turnRight.disabled = True
        b_stop.disabled = True
        b_backLeft.disabled = True
        b_backRight.disabled = True
        b_back.disabled = True
        s_speed.disabled = True

# widgets definitions

In [5]:
x_goal_widget = widgets.FloatText(
    value=0,
    description='x goal:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Insert X coordinate'],
   
)


y_goal_widget = widgets.FloatText(
    value=0,
    description='y goal:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Insert Y coordinate'],
   
)

toggle_mode = widgets.ToggleButtons(
    options=[('Cancel goal',0), ('Set new goal',1), ('Fully manual drive',2), ('Assisted Manual Drive',3)],
    description='Select Action:',
    disabled=False,
    value=0,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['cancel last given goal', 'make sure of selecting desired coordinates before setting the goal', 'manually drive the robot','manually drive the robot WITH assisted collision avoidance'],
   
    #     icons=['check'] * 3
)

InfoText = widgets.Text(
    value=' ',
    placeholder='',
    description='status:',
    disabled=True
)

b_forward = Button(description='FORWARD',
           layout=Layout(width='40%', height='65px' ))
b_forward.style.button_color = 'lightgreen'
b_forward.style.font_weight='bold'

b_stop = Button(description='STOP',
           layout=b_forward.layout)
b_stop.style.button_color = '#F44336' #red
b_stop.style.font_weight='bold'

b_forLeft = Button(description='LEFT FORWARD',
           layout=b_forward.layout)
b_forLeft.style.button_color = '#1565C0'
b_forLeft.style.font_weight='bold'

b_forRight = Button(description='RIGHT FORWARD',
           layout=b_forward.layout)
b_forRight.style.button_color = '#1565C0'
b_forRight.style.font_weight='bold'

b_turnLeft = Button(description='TURN LEFT',
           layout=b_forward.layout)
b_turnLeft.style.button_color = 'lightgreen'
b_turnLeft.style.font_weight='bold'

b_turnRight = Button(description='TURN RIGHT',
           layout=b_forward.layout)
b_turnRight.style.button_color = 'lightgreen'
b_turnRight.style.font_weight='bold'

b_backLeft = Button(description='BACK LEFT',
           layout=b_forward.layout)
b_backLeft.style.button_color = '#039BE5'
b_backLeft.style.font_weight='bold'

b_backRight = Button(description='BACK RIGHT',
           layout=b_forward.layout)
b_backRight.style.button_color = '#039BE5'
b_backRight.style.font_weight='bold'

b_back = Button(description='BACKWARDS',
           layout=b_forward.layout)
b_back.style.button_color = 'lightgreen'
b_back.style.font_weight='bold'

s_speed = widgets.FloatSlider(
    value=1,
    min=0.25,
    max=3,
    step=0.25,
    description='Speed multiplier factor:',
    disabled=True,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout = Layout(width='100%', height='65px')
)


first_row = HBox([b_forLeft,b_forward,b_forRight])
second_row= HBox([b_turnLeft,b_stop,b_turnRight])
third_row = HBox([b_backLeft,b_back,b_backRight])
commands = VBox([first_row,second_row,third_row])
DisableCommands(1)


# callback function to update goal_reached 

In [6]:
goal_reached = 0
goal_NON_reached = 0
def GoalStatusCallback(msg):
    global goal_reached
    global goal_NON_reached
    display(msg.data)
    if msg.data == False:
        goal_NON_reached = goal_NON_reached +1
        InfoText.value ='Goal UNREACHABLE'
        
    else:
        goal_reached = goal_reached +1
        InfoText.value ='Goal REACHED'
    
sub = rospy.Subscriber("my_reach", Bool, GoalStatusCallback)



# Widgets' behaviour definitions

In [7]:
def move_forward(arg):
    desiredVel.linear.x = 0.5*s_speed.value
    desiredVel.angular.z = 0.0
    pubManualDrive.publish(desiredVel)   
b_forward.on_click(move_forward) 


def stop(arg):
    desiredVel.linear.x = 0
    desiredVel.angular.z = 0
    pubManualDrive.publish(desiredVel)   
b_stop.on_click(stop) 


def move_for_left(arg):
    desiredVel.linear.x = 0.5*s_speed.value
    desiredVel.angular.z = 1
    pubManualDrive.publish(desiredVel)   
b_forLeft.on_click(move_for_left) 

def move_for_right(arg):
    desiredVel.linear.x = 0.5*s_speed.value
    desiredVel.angular.z = -1
    pubManualDrive.publish(desiredVel)   
b_forRight.on_click(move_for_right) 

def turn_left(arg):
    desiredVel.linear.x = 0
    desiredVel.angular.z = 1*s_speed.value
    pubManualDrive.publish(desiredVel)   
b_turnLeft.on_click(turn_left) 

def turn_right(arg):
    desiredVel.linear.x = 0
    desiredVel.angular.z = -1*s_speed.value
    pubManualDrive.publish(desiredVel)   
b_turnRight.on_click(turn_right) 


def move_back_left(arg):
    desiredVel.linear.x = -0.5*s_speed.value
    desiredVel.angular.z = -1
    pubManualDrive.publish(desiredVel)   
b_backLeft.on_click(move_back_left) 

def move_back_right(arg):
    desiredVel.linear.x = -0.5*s_speed.value
    desiredVel.angular.z = 1
    pubManualDrive.publish(desiredVel)   
b_backRight.on_click(move_back_right) 


def move_back(arg):
    desiredVel.linear.x = -0.5*s_speed.value
    desiredVel.angular.z = 0
    pubManualDrive.publish(desiredVel)   
b_back.on_click(move_back) 

In [8]:
def toggle_callback(mode):
        
        if (mode == 0):            
            my_cancel.id = "";
            pubCancel.publish(my_cancel);
            InfoText.value ='Goal Canceled'
            
            inputPoint.x = 666
            pubGoalPoint.publish(inputPoint);
            DisableGoal(0)
            DisableCommands(1)
            
        elif(mode == 1):
            DisableGoal(0)
            DisableCommands(1)
            InfoText.value ='new goal set'
            
            inputPoint.x = x_goal_widget.value
            inputPoint.y = y_goal_widget.value
            pubGoalPoint.publish(inputPoint)
            
        elif(mode == 2):
            DisableGoal(0)
            DisableCommands(0)
            InfoText.value ='Enabling fully manual drive'
            
            whichTeleop.data = 0
            pubTeleop.publish(whichTeleop);
            
        elif(mode == 3):
            DisableGoal(0)
            DisableCommands(0)
            InfoText.value ='Enabling Assisted manual drive'
            
            whichTeleop.data = 1
            pubTeleop.publish(whichTeleop);

        else:

            return

interactive_output (toggle_callback, {'mode':  toggle_mode})


Output()

# DISPLAY WIDGETS FOR MODE SELECTION AND GOAL SETTING

In [9]:

display(x_goal_widget, y_goal_widget)
display(toggle_mode)
display(InfoText)


FloatText(value=0.0, description='x goal:')

FloatText(value=0.0, description='y goal:')

ToggleButtons(description='Select Action:', options=(('Cancel goal', 0), ('Set new goal', 1), ('Fully manual d…

Text(value='Goal Canceled', description='status:', disabled=True, placeholder='')

# DISPLAY WIDGETS FOR MANUAL DRIVING

In [10]:
display(commands)
display(s_speed)

FloatSlider(value=1.0, description='Speed multiplier factor:', disabled=True, layout=Layout(height='65px', wid…

# 3D representation of robot's position

In [17]:
from jupyros import ros3d
import os
v = ros3d.Viewer()
rc = ros3d.ROSConnection(url="ws://localhost:9090")
tf_client = ros3d.TFClient(ros=rc, fixed_frame='map')

laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=rc, 
tf_client=tf_client)
urdf = ros3d.URDFModel(ros=rc, tf_client=tf_client, 
path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000'))
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

# Graphical representations about  the robot's output

In [12]:
%matplotlib widget
odom_vis = Odom_Visualiser()
subOdom = rospy.Subscriber('/odom', Odometry, odom_vis.odom_callback)
ani1 = FuncAnimation(odom_vis.fig, odom_vis.update_plot, init_func=odom_vis.plot_init,interval=1000, blit=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
goal_vis = Goal_Visualiser()
subGoal = rospy.Subscriber('/clock', Clock, goal_vis.goal_callback)
ani3= FuncAnimation(goal_vis.fig, goal_vis.update_plot, init_func=goal_vis.plot_init,interval=500, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
laser_vis = Laser_Visualiser()
subLaser = rospy.Subscriber('/scan', LaserScan, laser_vis.laser_callback)
ani2 = FuncAnimation(laser_vis.fig, laser_vis.update_plot, init_func=laser_vis.plot_init,interval=500, blit=False)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …